In [1]:
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
spark = get_spark_session(dev=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/27 13:14:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_namespace = 'gold' 
# spark.sql(f"DROP NAMESPACE IF EXISTS {test_namespace} CASCADE")
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}").show()

++
||
++
++



In [3]:
delivered_order_product = spark.read.table('warehousedev.gold.delivered_order_product')
delivered_order_product.show(n=5)

+--------------------+--------------------+---------------+-------------+--------------------+-----------+
|            order_id|          product_id|       category|order_item_id|           seller_id|total_price|
+--------------------+--------------------+---------------+-------------+--------------------+-----------+
|00010242fe8c5a6d1...|4244733e06e7ecb49...|     cool_stuff|            1|48436dade18ac8b2b...|      72.19|
|00018f77f2f0320c5...|e5f2d52b802189ee6...|       pet_shop|            1|dd7ddc04e1b6c2c61...|     259.83|
|000229ec398224ef6...|c777355d18b72b67a...|furniture_decor|            1|5b51032eddd242adc...|     216.87|
|00024acbcdf0a6daa...|7634da152a4610f15...|      perfumery|            1|9d7a1d34a50524090...|      25.78|
|00042b26cf59d7ce6...|ac6c3623068f30de0...|   garden_tools|            1|df560393f3a51e745...|     218.04|
+--------------------+--------------------+---------------+-------------+--------------------+-----------+
only showing top 5 rows



In [4]:
# gold
sale_stats = delivered_order_product.groupBy('product_id', 'category', 'seller_id').agg(
    F.count(F.col('order_id')).alias('order_count'),
    F.round(F.sum(F.col('total_price')), 4).alias('total_sales')
    ).orderBy(F.col('order_count').desc())

sale_stats = sale_stats.withColumn('mean_sale', F.round(F.col('total_sales') / F.col('order_count'), 4))
sale_stats.show()

+--------------------+--------------------+--------------------+-----------+-----------+---------+
|          product_id|            category|           seller_id|order_count|total_sales|mean_sale|
+--------------------+--------------------+--------------------+-----------+-----------+---------+
|aca2eb7d00ea1a7b8...|     furniture_decor|955fee9216a65b617...|        520|   44197.75|  84.9957|
|422879e10f4668299...|        garden_tools|1f50f920176fa81da...|        484|   34201.26|  70.6638|
|99a4788cb24856965...|      bed_bath_table|4a3ca9315b744ce9f...|        471|   49338.25| 104.7521|
|389d119b48cf3043d...|        garden_tools|1f50f920176fa81da...|        390|   28543.65|  73.1888|
|368c6c730842d7801...|        garden_tools|1f50f920176fa81da...|        388|    27984.4|  72.1247|
|53759a2ecddad2bb8...|        garden_tools|1f50f920176fa81da...|        373|   27268.23|  73.1052|
|d1c427060a0f73f6b...|computers_accesso...|a1043bafd471dff53...|        332|   58957.31| 177.5823|
|154e7e31e

In [5]:
sale_stats.filter(F.col('product_id') == 'bb50f2e236e5eea0100680137654686c').show()

+--------------------+-------------+--------------------+-----------+-----------+---------+
|          product_id|     category|           seller_id|order_count|total_sales|mean_sale|
+--------------------+-------------+--------------------+-----------+-----------+---------+
|bb50f2e236e5eea01...|health_beauty|f7ba60f8c3f99e7ee...|        194|   67258.03| 346.6909|
+--------------------+-------------+--------------------+-----------+-----------+---------+



In [6]:
sale_stats.writeTo(f"{test_namespace}.sale_stats").createOrReplace()

In [7]:
spark.stop()